# Building Data_model for stock price prediction ETL

## Step 1: Select stocks to predict tommorrow's price 

As a example, here I will use a list of five stocks from different industry/sector

In [104]:
from pyspark.sql import SparkSession
import os
import configparser
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
sc.install_pypi_package("yfinance") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [92]:
sc.install_pypi_package("pyarrow") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [93]:
stocks=['AAPL', 'INTC', 'TSLA', 'GILD', 'BA']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Step 2: Pull stocks data from yahoo finance and save to dataframes

In [94]:
import pandas as pd
import yfinance as yf
import pyarrow
import datetime

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
def download_current_stocks_to_df(stocks, ndays):
    end=datetime.datetime.now().strftime("%Y-%m-%d")
    start=(datetime.datetime.now()-datetime.timedelta(days=ndays)).strftime("%Y-%m-%d")
    stock_dict={}
    for symbol in stocks:
        df_symbol=yf.download(symbol,start,end,progress=False)
        stock_dict.update({symbol:df_symbol})
    return stock_dict


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
stock_dict=download_current_stocks_to_df(stocks, 30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Step 3: scrape top news data of today or before market open

It is recommend to schedule the job just a little before market open (i.e. 9:30AM ET) to obtain the most recent news

In [10]:
sc.install_pypi_package("praw") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/5c/39/17251486951815d4514e4a3f179d4f3e7af5f7b1ce8eaba5a3ea61bc91f2/praw-7.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c9/8e/d076cb8f26523f91eef3e75d6cf9143b2f16d67ce7d681a61d0bbc783f49/prawcore-1.3.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4c/5f/f61b420143ed1c8dc69f9eaec5ff1ac36109d52c80de49d66e0c36c3dfdf/websocket_client-0.57.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d6/c3/aaf8a162df8e8f9d321237c7c0e63aff95b42d19f1758f96606e3cabb245/update_checker-0.17-py2.py3-none-any.whl

In [11]:
#! usr/bin/env python3
import praw
import pandas as pd
import datetime as dt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
reddit = praw.Reddit(client_id='FbxkAEhvb7pxbg', \
                     client_secret='zyl8WnU9X5rr7y5_EeGUVj994kA', \
                     user_agent='topnewsscraper', \
                     username='xintao0202', \
                     password='zangma53@198422')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
subreddit = reddit.subreddit('worldnews')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
top_subreddit = subreddit.top("day")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
topics_dict = { "title":[], "score":[], "created": []}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
#     topics_dict["id"].append(submission.id)
#     topics_dict["url"].append(submission.url)
#     topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(dt.datetime.fromtimestamp(submission.created))
#     topics_dict["body"].append(submission.selftext)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
topics_df = pd.DataFrame(topics_dict)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
pd.set_option('max_columns', None)
topics_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                               title  score  \
0  The U.S. has certified that Hong Kong is no lo...  91856   
1  Rio Tinto legally destroy Native Australian Ab...  35242   
2  France bans use of hydroxychloroquine to cure ...   9240   
3  Hong Kong police fire pepperballs at protester...   4324   
4  Boris Johnson is tumbling in the polls after i...   4087   

              created  
0 2020-05-27 23:54:08  
1 2020-05-28 06:27:39  
2 2020-05-27 17:54:55  
3 2020-05-27 15:33:52  
4 2020-05-28 01:10:48

## Step 4 Process stock and news data, and save thme to S3

This part of the data is for prediction. Once we trained a model, this is for predict. Instead of joining the dataframes together, it is better to keep them separated for DB storage and further analysis. The most current data will be pulled daily using Airflow, save them to S3 as well as load to Redshift cluster

Similary as historical data, we will add technical indicators to stock data. The reason why we don't just pull one day stock is to calculate the technical indicators, which needs n days. Here I set n<30 

In [97]:
def get_bbands(df, ndays):
    dm = df[['Close']].rolling(ndays).mean()
    ds = df[['Close']].rolling(ndays).std()
    df['upperBB'] = dm + 2 * ds
    df['lowerBB'] = dm - 2 * ds
    return df

# Simple Moving Average
def get_SMA(df, ndays):
    df['SMA']=df[['Close']].rolling(ndays).mean()
    return df

# Expontential Moving Average
def get_EMA(df, ndays):
    df['EMA'] = df[['Close']].ewm( span = ndays, min_periods = ndays - 1).mean()
    return df

# Rate of Change
def get_ROC(df, ndays):
    dn = df[['Close']].diff(ndays)
    dd = df[['Close']].shift(ndays)
    df['ROC'] = dn/dd
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
def save_to_s3(df, stock, folder):
    df_copy=df.copy()
    df_copy.reset_index(inplace=True)
    s3 = boto3.resource('s3')
    json_data=df_copy.to_json(orient='records')
    key="{}/{}.json".format(folder, stock)
    s3object = s3.Object('stock.etl', key)
    s3object.put(Body=(bytes(json.dumps(json_data).encode('UTF-8'))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
import numpy as np
import json

def transformation_save_stock(stock_df, stock_symbol):
    # First Keep useful information from stock price tables (Date, close and volume), forward fill missing values. 
    stock_price_df=stock_df.copy()
    stock_price_df.drop(['Open','High','Low', 'Close'], axis=1, inplace=True)
    stock_price_df.fillna(method='ffill')
    stock_price_df.rename(columns={'Adj Close':'Close'}, inplace=True)
    
    
    # Adding daily return (%), stock_rise (1, 0 if drop), technical indicators 
    stock_price_df['daily_return']=stock_price_df[['Close']]/stock_price_df[['Close']].shift(1)-1
    stock_price_df=get_bbands(stock_price_df, 10)
    stock_price_df=get_SMA(stock_price_df, 10)
    stock_price_df=get_EMA(stock_price_df, 10)
    stock_price_df=get_ROC(stock_price_df, 1)
    stock_price_df['stock_rise']=np.where(stock_price_df['daily_return']>0, 1, 0)
    
    # save stock data
    today_date=datetime.datetime.now().strftime("%Y.%m.%d")
    save_to_s3(stock_price_df, stock_symbol, "current/{}".format(today_date))
    
def transformation_save_news(news_df):      
   # process news data
    news_df_copy=news_df.copy()
    news_df_copy['Rank']=news_df_copy['score'].rank(method='dense', ascending=False).astype(int)
    news_df_copy.columns=['News', 'Score', 'Date', 'Rank']
    news_df_copy=news_df_copy[['Date', 'Rank', 'Score', 'News']] 
    
    # save news data
    save_to_s3(news_df_copy, "24hrNews" "current/{}".format(today_date))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##  Data Quality checks

First theck the output of the historic ETL. The result should be 7163 files (generated from 7163 stocks in raw zip file- stock folder) and the size of the smallest file should be larger than 0B

In [108]:
import os
import commands


def data_quality_check(folder, n_files):
    output=os.system("aws s3 ls s3://stock.etl/{}/ --recursive | wc -l".format(folder))
    if (output!=n_files):
        raise ValueError('Data quality check failed: file number not match')
    status, output = commands.getstatusoutput("aws s3api list-objects-v2 --bucket stock.etl --prefix {} --output text --query 'sort_by(Contents,&Size)[0].Size'".format(folder))
    if (int(output[0]))<=0:
        raise ValueError('Data quality check failed: contains empty file')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Function used for DAGs


Copy the functions below for DAG python file. 

In [102]:
import os
import pandas as pd
import yfinance as yf
import pyarrow
import datetime
import praw
import pandas as pd
import numpy as np
import json

def download_current_stocks_to_df(stocks, ndays):
    end=datetime.datetime.now().strftime("%Y-%m-%d")
    start=(datetime.datetime.now()-datetime.timedelta(days=ndays)).strftime("%Y-%m-%d")
    stock_dict={}
    for symbol in stocks:
        df_symbol=yf.download(symbol,start,end,progress=False)
        stock_dict.update({symbol:df_symbol})
    return stock_dict

def get_24hr_news():
    reddit = praw.Reddit(client_id='FbxkAEhvb7pxbg', \
                     client_secret='zyl8WnU9X5rr7y5_EeGUVj994kA', \
                     user_agent='topnewsscraper', \
                     username='xintao0202', \
                     password='zangma53@198422')
    subreddit = reddit.subreddit('worldnews')
    top_subreddit = subreddit.top("day")
    topics_dict = { "title":[], "score":[], "created": []}
    
    for submission in top_subreddit:
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
    #     topics_dict["id"].append(submission.id)
    #     topics_dict["url"].append(submission.url)
    #     topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(datetime.datetime.fromtimestamp(submission.created))
    #     topics_dict["body"].append(submission.selftext)
    
    topics_df = pd.DataFrame(topics_dict)
    return topics_df

def transformation_save_stock(stock_df, stock_symbol):
    # First Keep useful information from stock price tables (Date, close and volume), forward fill missing values. 
    stock_price_df=stock_df.copy()
    stock_price_df.drop(['Open','High','Low', 'Close'], axis=1, inplace=True)
    stock_price_df.fillna(method='ffill')
    stock_price_df.rename(columns={'Adj Close':'Close'}, inplace=True)
    
    
    # Adding daily return (%), stock_rise (1, 0 if drop), technical indicators 
    stock_price_df['daily_return']=stock_price_df[['Close']]/stock_price_df[['Close']].shift(1)-1
    stock_price_df=get_bbands(stock_price_df, 10)
    stock_price_df=get_SMA(stock_price_df, 10)
    stock_price_df=get_EMA(stock_price_df, 10)
    stock_price_df=get_ROC(stock_price_df, 1)
    stock_price_df['stock_rise']=np.where(stock_price_df['daily_return']>0, 1, 0)
    
    # save stock data
    today_date=datetime.datetime.now().strftime("%Y.%m.%d")
    save_to_s3(stock_price_df, stock_symbol, "current/stocks/{}".format(today_date))
    
def transformation_save_news(news_df):      
   # process news data
    news_df_copy=news_df.copy()
    news_df_copy['Rank']=news_df_copy['score'].rank(method='dense', ascending=False).astype(int)
    news_df_copy.columns=['News', 'Score', 'Date', 'Rank']
    news_df_copy=news_df_copy[['Date', 'Rank', 'Score', 'News']] 
    
    # save news data
    save_to_s3(news_df_copy, "24hrNews" "current/news/{}".format(today_date))

def current_stocks_etl(list_of_stocks, ndays):
    stocks_dict=download_current_stocks_to_df(list_of_stocks, ndays)
    for key, value in stocks_dict.items():
        stock_name=key.upper()
        transformation_save_stock(value, stock_name)
    
def current_news_etl():
    topics_df=get_24hr_news()
    transformation_save_news(topics_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…